# OpenAI Assistant V2 - RAG 테스트

- 저작자: [테디노트](https://www.youtube.com/@teddynote)

**코드 재배포시 출처를 남겨 주세요**

**OpenAI API KEY 발급 방법**

- [OpenAI 키 발급방법](https://wikidocs.net/233342)

`.env` 에 OpenAI API KEY 를 저장해 두었다면 밑의 코드 주석을 해제하고 실행하세요.

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')


## 주요 링크

- [OpenAI Assistant 대시보드](https://platform.openai.com/assistants)
- [OpenAI API KEY 발급](https://platform.openai.com/api-keys)
- [OpenAI Files 스토리지](https://platform.openai.com/storage/files)

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

## STEP 1) 설정

- `_DEFAULT_RAG_INSTRUCTIONS` 는 RAG 의 instruction 입니다. 프롬프트를 자유롭게 수정하여 테스트해 보세요.
- `OPENAI_API_KEY` 는 발급받은 OPENAI API KEY 를 입력해 주세요.
- `PROJECT_NAME` 은 프로젝트 이름을 입력해 주세요.
- `model_name` 은 사용할 모델을 입력해 주세요. (예, `gpt-4o`, `gpt-4o-mini`, ...)
- `chunk_size` 는 청크 크기를 입력해 주세요.
- `chunk_overlap` 는 청크 중복 크기를 입력해 주세요.

In [4]:
from langchain_teddynote.models import OpenAIAssistant


# RAG 시스템 프롬프트 입력
_DEFAULT_RAG_INSTRUCTIONS = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Answer in Korean."""


# 설정(configs)
configs = {
    "OPENAI_API_KEY": openai_api_key,  # OpenAI API 키
    "instructions": _DEFAULT_RAG_INSTRUCTIONS,  # RAG 시스템 프롬프트
    "PROJECT_NAME": "PDF-RAG-TEST",  # 프로젝트 이름(자유롭게 설정)
    "model_name": "gpt-4o",  # 사용할 OpenAI 모델 이름(gpt-4o, gpt-4o-mini, ...)
    "chunk_size": 1000,  # 청크 크기
    "chunk_overlap": 100,  # 청크 중복 크기
}


# 인스턴스 생성
assistant = OpenAIAssistant(configs)

## STEP 2) 파일 업로드

### 파일 업로드 하지 않은 경우

- RAG 할 파일을 업로드 합니다.
- 파일 업로드는 딱 1번만 하면 됩니다.

이미 파일을 업로드 하였고, file_id 가 있다면, 아래 코드는 생략합니다.

In [7]:
# 업로드할 파일 경로
data = ".\data\SPRI_AI_Brief_2023년12월호.pdf"

# 파일 업로드 후 file_id 는 잘 보관해 두세요. (대시보드에서 나중에 확인 가능)
file_id = assistant.upload_file(data)

## STEP 3-1) Assistant, VectorStore 설정

(참고) 이미 있는 assistant_id, vector_id 가 있다면, `STEP 3-2` 으로 넘어가세요.

- `assistant_id`, `vector_id` 는 잘 보관해 두세요 (메모장 같은 곳에...)

In [8]:
# 업로드한 파일의 ID 리스트 생성
file_ids = [file_id]

# 새로운 어시스턴트 생성 및 ID 받기
assistant_id, vector_id = assistant.create_new_assistant(file_ids)

# 어시스턴트 설정
assistant.setup_assistant(assistant_id)

# 벡터 스토어 설정
assistant.setup_vectorstore(vector_id)

## STEP 3-2) 이미 있는 assistant_id, vector_id 가 있다면, 아래와 같이 설정합니다.

만약에 이미 있는 assistant_id, vector_id 가 있다면, 아래와 같이 설정합니다.

In [ ]:
# assistant_id = "asst_TdJbouMO7gU........."
# vector_id = "vs_ibSoSSErFDnS9........"

# # 어시스턴트 설정
# assistant.setup_assistant(assistant_id)

# # 벡터 스토어 설정
# assistant.setup_vectorstore(vector_id)

## STEP 4) 대화

- `stream()` 스트리밍 기능을 지원합니다.
- `invoke()` 는 일반 출력입니다. 답변 생성이 끝날때까지 기다렸다가 한 번에 출력합니다.

`stream()` 스트리밍 기능을 지원합니다.

In [9]:
for token in assistant.stream("삼성전자가 개발한 생성형 AI의 이름은?"):
    print(token, end="", flush=True)

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【4:0†source】.

아래는 좀 더 편리하게 스트리밍 출력을 해줍니다. (단, 직접 토큰을 받아서 화면에 뿌려야 하는 경우에는 이전 코드가 더 유용합니다.)

In [10]:
from langchain_teddynote.messages import stream_response

stream_response(assistant.stream("이전 답변을 영어로"))

The generative AI developed by Samsung Electronics is called 'Samsung GAUSS'【4:0†source】.

`invoke()` 는 일반 출력입니다. 답변 생성이 끝날때까지 기다렸다가 한 번에 출력합니다.

In [11]:
# 질문
print(assistant.invoke("삼성전자가 개발한 생성형 AI의 이름은?"))

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【10:0†SPRI_AI_Brief_2023년12월호.pdf】.


In [12]:
# 추가 질문
print(assistant.invoke("이전의 답변에 대한 출처(페이지)를 알려줘"))

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【14:0†source】.


대화 목록을 조회합니다.

In [13]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '삼성전자가 개발한 생성형 AI의 이름은?', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【4:0†source】.",
  'role': 'assistant'},
 {'content': '이전 답변을 영어로', 'role': 'user'},
 {'content': "The generative AI developed by Samsung Electronics is called 'Samsung GAUSS'【4:0†source】.",
  'role': 'assistant'},
 {'content': '삼성전자가 개발한 생성형 AI의 이름은?', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【10:0†SPRI_AI_Brief_2023년12월호.pdf】.",
  'role': 'assistant'},
 {'content': '이전의 답변에 대한 출처(페이지)를 알려줘', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【14:0†source】.",
  'role': 'assistant'}]

새로운 주제는 새로운 대화 쓰레드(thread)를 시작해야 합니다.

In [14]:
# 대화 초기화
assistant.clear_chat_history()

새로운 주제의 대화를 시작합니다.

In [15]:
# 질문
stream_response(
    assistant.stream("구글이 앤쓰로픽에 투자 계획을 발표했습니다. 이에 대해 알려줘")
)

구글이 앤쓰로픽에 투자 계획을 발표했으며, 그 내용은 다음과 같습니다:

구글은 2023년 10월 27일 앤쓰로픽에 최대 20억 달러를 투자하기로 합의하고, 초기 5억 달러를 우선 투자했습니다. 앤쓰로픽은 구글과 클라우드 서비스 사용 계약도 체결했습니다【4:0†source】. 이는 구글이 2023년 2월에 이미 앤쓰로픽에 5억 5천만 달러를 투자한 것에 이어지는 것입니다【4:0†source】.

구글, 마이크로소프트, 아마존 등 주요 클라우드 사업자들은 차세대 AI 모델의 대표 기업인 앤쓰로픽 및 오픈AI와 협력을 확대하는 추세에 있습니다【4:0†source】. 여기에는 앤쓰로픽이 개발한 챗GPT의 경쟁자인 '클로드(Claude)'와 같은 대형 언어 모델도 포함됩니다【4:0†source】.

추가적으로, 구글은 앤쓰로픽의 클라우드 서비스 사용을 위해 4년 간 총 30억 달러 규모의 계약을 체결했습니다【4:0†source】.

In [16]:
# 추가 질문
stream_response(assistant.stream("이전의 답변을 보고서 양식으로 작성해 주세요"))

### 구글의 앤쓰로픽(Anthropic) 투자 계획 보고서

#### 1. 개요
2023년 10월 27일, 구글은 인공지능(AI) 연구 및 개발 기업인 앤쓰로픽에 대한 대규모 투자 계획을 발표했습니다. 이 보고서는 구글의 투자 배경, 투자 규모, 그리고 앤쓰로픽과의 협력 내용을 중점적으로 다룹니다.

#### 2. 투자 배경
구글, 마이크로소프트, 아마존 등 주요 클라우드 사업자들은 AI 모델의 혁신을 주도하는 기업들과의 협력을 통해 차세대 AI 기술 개발을 가속화하고 있습니다. 구글은 이러한 전략의 일환으로, 앤쓰로픽과의 협력을 결정하였으며, 이는 구글의 AI 연구 역량 강화와 클라우드 서비스 시장에서의 경쟁력을 높이기 위한 목적을 가지고 있습니다【4:0†source】.

#### 3. 투자 규모 및 조건
- **최대 투자액**: 구글은 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했습니다.
- **초기 투자액**: 우선 초기 5억 달러를 투자하였습니다【4:0†source】.
- **이전 투자**: 2023년 2월에 이미 5억 5천만 달러를 투자한 바 있습니다【4:0†source】.
- **클라우드 서비스 계약**: 구글과 앤쓰로픽은 구글 클라우드의 사용을 포함한 4년 간 총 30억 달러 규모의 계약을 체결했습니다【4:0†source】.

#### 4. 협력 내용
앤쓰로픽은 구글과의 협력을 통해 자사의 AI 모델인 '클로드(Claude)'를 비롯한 주요 AI 기술 개발을 가속화할 예정입니다. 클로드는 OpenAI의 챗GPT와 경쟁하는 대형 언어 모델로, 이번 투자를 통해 연구 및 개발 역량을 강화할 것입니다【4:0†source】.

#### 5. 결론
구글과 앤쓰로픽 간의 이번 협력은 AI 기술 개발에서의 중요한 이정표로서, 양사의 기술적 시너지를 통한 혁신적 성과가 기대됩니다. 구글의 대규모 투자는 앤쓰로픽의 연구 역량을 강화하고, 전 세계 AI 및 클라우드 서비스 시장에서의 경쟁력을 제고하는 데 기여할 것입니다.

#### 6. 참고 문헌
- 구글과 앤

In [17]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '구글이 앤쓰로픽에 투자 계획을 발표했습니다. 이에 대해 알려줘', 'role': 'user'},
 {'content': "구글이 앤쓰로픽에 투자 계획을 발표했으며, 그 내용은 다음과 같습니다:\n\n구글은 2023년 10월 27일 앤쓰로픽에 최대 20억 달러를 투자하기로 합의하고, 초기 5억 달러를 우선 투자했습니다. 앤쓰로픽은 구글과 클라우드 서비스 사용 계약도 체결했습니다【4:0†source】. 이는 구글이 2023년 2월에 이미 앤쓰로픽에 5억 5천만 달러를 투자한 것에 이어지는 것입니다【4:0†source】.\n\n구글, 마이크로소프트, 아마존 등 주요 클라우드 사업자들은 차세대 AI 모델의 대표 기업인 앤쓰로픽 및 오픈AI와 협력을 확대하는 추세에 있습니다【4:0†source】. 여기에는 앤쓰로픽이 개발한 챗GPT의 경쟁자인 '클로드(Claude)'와 같은 대형 언어 모델도 포함됩니다【4:0†source】.\n\n추가적으로, 구글은 앤쓰로픽의 클라우드 서비스 사용을 위해 4년 간 총 30억 달러 규모의 계약을 체결했습니다【4:0†source】.",
  'role': 'assistant'},
 {'content': '이전의 답변을 보고서 양식으로 작성해 주세요', 'role': 'user'},
 {'content': "### 구글의 앤쓰로픽(Anthropic) 투자 계획 보고서\n\n#### 1. 개요\n2023년 10월 27일, 구글은 인공지능(AI) 연구 및 개발 기업인 앤쓰로픽에 대한 대규모 투자 계획을 발표했습니다. 이 보고서는 구글의 투자 배경, 투자 규모, 그리고 앤쓰로픽과의 협력 내용을 중점적으로 다룹니다.\n\n#### 2. 투자 배경\n구글, 마이크로소프트, 아마존 등 주요 클라우드 사업자들은 AI 모델의 혁신을 주도하는 기업들과의 협력을 통해 차세대 AI 기술 개발을 가속화하고 있습니다. 구글은 이러한 전략의 일환으로, 앤쓰로픽과의 협력을 결정하였으며, 이는 구글의 AI 연구 역량 강화와 클라우드 서비스 시장에